In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from binance.client import Client
import pandas as pd
import requests
import pandas as pd
import hopsworks
import os
import warnings
warnings.filterwarnings("ignore")

/opt/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
os.environ["HOPSWORKS_API_KEY"] = "WMg6iOTOV75YfUcb.AcuNGWjcMS79s4bz0FqZsRaJtcU8Siqu7Mx084dfkRHauMxK251VTYLYSBAIqBb5"
project = hopsworks.login()
fs = project.get_feature_store() 
fg = fs.get_feature_group('data_historic', version=1)
df = fg.read()

2025-01-05 17:53:54,616 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-05 17:53:54,622 INFO: Initializing external client
2025-01-05 17:53:54,622 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-05 17:53:56,104 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1168537
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.19s) 


In [5]:
df.head()

,timestamp,inflation_rate,open,high,low,gold_price,volume
0,2024-07-01 01:00:00,0.083721,2310.0,2319.0,2308.0,2317.0,105.7411
1,2024-03-14 12:00:00,-0.109840,2150.0,2152.0,2145.0,2147.0,34.5638
2,2024-05-14 13:00:00,0.129594,2331.0,2337.0,2328.0,2337.0,34.2310
3,2024-10-03 19:00:00,0.000000,2666.0,2668.0,2663.0,2664.0,28.3198
4,2024-10-27 16:00:00,0.018524,2764.0,2771.0,2763.0,2767.0,41.9252


In [6]:
# shifted_c = ['inflation_rate', 'open', 'high','low', 'volume']
# df[shifted_c] = df[shifted_c].shift(1)
# df.dropna(inplace=True)
# print(df.head())

In [7]:
data_view = fs.get_or_create_feature_group(
    name='data_for_view',
    description='data view',
    version=1,
    primary_key=['timestamp'],
)
data_view.insert(df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1168537/fs/1159240/fg/1394602


Uploading Dataframe: 100.00% |██████████| Rows 8760/8760 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: data_for_view_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1168537/jobs/named/data_for_view_1_offline_fg_materialization/executions


(Job('data_for_view_1_offline_fg_materialization', 'SPARK'), None)

In [8]:
data_fg = fs.get_feature_group(
    name='data_for_view',
    version=1,
)


In [15]:
features = data_fg.select_all()
features.show(10)


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.08s) 


,timestamp,inflation_rate,open,high,low,gold_price,volume
0,2024-07-01 01:00:00,0.083721,2310.0,2319.0,2308.0,2317.0,105.7411
1,2024-03-14 12:00:00,-0.109840,2150.0,2152.0,2145.0,2147.0,34.5638
2,2024-05-14 13:00:00,0.129594,2331.0,2337.0,2328.0,2337.0,34.2310
3,2024-10-03 19:00:00,0.000000,2666.0,2668.0,2663.0,2664.0,28.3198
4,2024-10-27 16:00:00,0.018524,2764.0,2771.0,2763.0,2767.0,41.9252
5,2024-02-07 12:00:00,0.009285,2020.0,2021.0,2018.0,2020.0,9.0567
6,2024-06-21 09:00:00,0.028066,2342.0,2349.0,2339.0,2343.0,43.0758
7,2024-10-07 23:00:00,-0.027337,2659.0,2660.0,2656.0,2657.0,19.9857
8,2024-09-07 14:00:00,0.018085,2503.0,2507.0,2503.0,2503.0,4.1544
9,2024-09-16 19:00:00,0.026954,2588.0,2590.0,2587.0,2588.0,26.9475


In [16]:
feature_view = fs.get_or_create_feature_view(
    name='gold_prices_fv',
    description="gold_price",
    version=1,
    labels=['gold_price'],
    query=features,
)

In [17]:
X_train, X_test, y_train, y_test = feature_view.train_test_split(test_size=0.1)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.08s) 


In [18]:
X_train

,timestamp,inflation_rate,open,high,low,volume
0,2024-07-01 01:00:00,0.083721,2310.0,2319.0,2308.0,105.7411
1,2024-03-14 12:00:00,-0.109840,2150.0,2152.0,2145.0,34.5638
2,2024-05-14 13:00:00,0.129594,2331.0,2337.0,2328.0,34.2310
3,2024-10-03 19:00:00,0.000000,2666.0,2668.0,2663.0,28.3198
4,2024-10-27 16:00:00,0.018524,2764.0,2771.0,2763.0,41.9252
...,...,...,...,...,...,...
8755,2024-05-03 14:00:00,-0.102192,2287.0,2295.0,2280.0,84.8458
8756,2024-11-18 07:00:00,0.000000,2590.0,2594.0,2577.0,85.9545
8757,2024-07-08 03:00:00,0.073978,2355.0,2357.0,2352.0,9.3620
8758,2024-09-08 04:00:00,-0.009046,2500.0,2501.0,2497.0,7.6072


In [19]:
train_features = X_train.drop(['timestamp'], axis=1)
test_features = X_test.drop(['timestamp'], axis=1)

In [20]:
from xgboost import XGBRegressor
my_model = XGBRegressor()
my_model.fit(train_features, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [23]:
y_pred = my_model.predict(test_features)
mse = mean_squared_error(y_test, y_pred)
print("The MSE is :", mse)


The MSE is : 16.765275452745286
